<a href="https://colab.research.google.com/github/deekshith7107/Aiden_hack/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required libraries
!pip install -q transformers datasets peft accelerate wandb bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 852.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U -q bitsandbytes>=0.43.0

In [5]:
!pip install -U -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 23.0 MB/s eta 0:00:00


In [6]:
import bitsandbytes
print(bitsandbytes.__version__)  # Should show ≥0.43.0

0.45.2


In [7]:
!pip install -U -q bitsandbytes-cuda121  # For CUDA 12.1

ERROR: Could not find a version that satisfies the requirement bitsandbytes-cuda121 (from versions: none)
ERROR: No matching distribution found for bitsandbytes-cuda121


In [8]:
!nvcc --version  # Should show CUDA ≥11.8

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [10]:
# Import necessary libraries
from google.colab import files
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    pipeline
)
from datasets import load_dataset, Dataset
from peft import LoraConfig, get_peft_model
import torch
import wandb
import os
import pandas as pd

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
cannot import name 'is_decord_available' from 'transformers.utils' (/usr/local/lib/python3.11/dist-packages/transformers/utils/__init__.py)

In [ ]:
# Configuration - Set your parameters here
BASE_MODEL = "gpt2"  # Try "EleutherAI/gpt-neo-125M" for larger models
USE_RLHF = False      # Set to True for RLHF training
USE_LORA = True       # Use LoRA for efficient training
EPOCHS = 3            # Number of training epochs
LEARNING_RATE = 2e-5  # Learning rate
BATCH_SIZE = 2        # Reduce if you get OOM errors
WANDB_KEY = ""        # Optional Weights & Biases key

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

def load_model(model_name):
    # Configure 4-bit quantization
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )


    # Load model with quantization
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer



def format_dataset(dataset):
    required_columns = {'prompt', 'completion'}
    # if not required_columns.issubset(dataset.column_names):
    #     raise ValueError("Dataset must contain 'prompt' and 'completion' columns")

    return dataset.map(lambda x: {
        "text": f"Instruction: {x['instruction']}\nResponse: {x['response']}"
    })

In [ ]:
# # Upload your dataset (CSV or JSON file with 'prompt' and 'completion' columns)
# print("Upload your dataset (CSV or JSON file with 'prompt' and 'completion' columns)")
# uploaded = files.upload()
# file_name = next(iter(uploaded))
# dataset_path = f"/content/{file_name}"
# Load a random dataset from Hugging Face
dataset = load_dataset("databricks/databricks-dolly-15k", split="train[:100]")  # Use 100 samples for testing
dataset = format_dataset(dataset)
dataset = dataset.train_test_split(test_size=0.1)

# Display a sample
print("Sample from the dataset:")
print(dataset['train'][0]['text'])

In [ ]:
# # Load and prepare dataset
# file_ext = file_name.split('.')[-1]
# dataset = load_dataset(file_ext, data_files=dataset_path)['train']
# dataset = format_dataset(dataset)
# dataset = dataset.train_test_split(test_size=0.1)

# print(f"Dataset sample:\n{dataset['train'][0]['text']}")
# Load a random dataset from Hugging Face
dataset = load_dataset("databricks/databricks-dolly-15k", split="train[:100]")  # Use 100 samples for testing
dataset = format_dataset(dataset)
dataset = dataset.train_test_split(test_size=0.1)

# Display a sample
print("Sample from the dataset:")
print(dataset['train'][0]['text'])

In [ ]:
# Initialize model and tokenizer
model, tokenizer = load_model(BASE_MODEL)

# Apply LoRA if enabled
if USE_LORA:
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()

In [ ]:
# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    logging_dir="./logs",
    report_to="wandb" if WANDB_KEY else None,
    save_strategy="epoch",
    evaluation_strategy="epoch"
)

In [ ]:
# Standard fine-tuning
if not USE_RLHF:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer
    )

    if WANDB_KEY:
        wandb.login(key=WANDB_KEY)

    trainer.train()
    model.save_pretrained("./fine_tuned_model")
    print("Standard fine-tuning completed!")

In [ ]:
# Inference test
def generate_response(prompt, model, tokenizer, max_length=200):
    inputs = tokenizer(f"Instruction: {prompt}\nResponse:", return_tensors="pt").to("cuda")
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test with sample prompt
test_prompt = "How do I open a bank account?"
print(generate_response(test_prompt, model, tokenizer))

In [ ]:
# Save model to Google Drive (optional)
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./fine_tuned_model /content/drive/My\ Drive/

In [ ]:
# Push model to Hugging Face Hub (optional)
from huggingface_hub import notebook_login
notebook_login()
model.push_to_hub("your-username/finetuned-model")